In [1]:
print("hi")


hi


In [2]:
%pwd

'c:\\ProgramData\\ANURITI\\Medical-Chatbot-\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\ProgramData\\ANURITI\\Medical-Chatbot-'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
from langchain.document_loaders import PyPDFLoader

def load_pdf_file(data):
    loader = PyPDFLoader(data)  # Use PyPDFLoader directly for a single file
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_file(data='C:/Users/ANURITI/Downloads/Medical_book.pdf')



In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [11]:
#text_chunks

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\ANURITI\AppData\Local\Temp\ipykernel_20244\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
#query_result

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

In [20]:

import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [23]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("What is Acne?")

In [27]:
retrieved_docs

[]

In [37]:
from langchain_groq import ChatGroq



In [38]:
import os
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"), 
    temperature=0.4, 
    max_tokens=500
)


In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [40]:
# Fix JSON serialization issue by using string messages
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [41]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [42]:
# Test Queries
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a hormonal disorder that develops when the pituitary gland produces too much growth hormone after the growth plates in bones have closed. Gigantism is a similar condition, but it occurs before the growth plates close, causing excessive growth and tall stature. Both conditions can lead to various health issues, such as joint pain and heart problems.


In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

Stats, short for statistics, is a branch of mathematics that deals with the collection, analysis, interpretation, and presentation of data. It involves the use of mathematical theories and methods to draw meaningful conclusions from data and make informed decisions. Statistics can be applied in various fields, including business, economics, engineering, medicine, and social sciences.


: 